In [1]:
%pip install arango-datasets
%pip install nx-arangodb

%pip install --upgrade --quiet  python-arango # The ArangoDB Python Driver
!pip install gradio
%pip install --upgrade --quiet  langchain_google_vertexai
%pip install --upgrade --quiet  langchain_community
%pip install --upgrade --quiet  langchain-google-genai
!pip install langchain
!pip install langgraph
# Install RAPIDS cuGraph
!pip install nx-cugraph-cu12 --extra-index-url https://pypi.nvidia.com

  Using cached gradio-5.20.1-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached fastapi-0.115.11-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached gradio_client-1.7.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached MarkupSafe-2.1.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%env NX_CUGRAPH_AUTOCONFIG=True
import networkx

env: NX_CUGRAPH_AUTOCONFIG=True


In [3]:
# If your API Keys are in Colab Secrets
import sys
import os

if "google.colab" in sys.modules:
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")


In [4]:
PROJECT_ID = "gem-creation"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "true"

In [5]:
from google import genai

client = genai.Client()
client

In [6]:
if not client.vertexai:
    print(f"Using Gemini Developer API.")
elif client._api_client.project:
    print(
        f"Using Vertex AI with project: {client._api_client.project} in location: {client._api_client.location}"
    )
elif client._api_client.api_key:
    print(
        f"Using Vertex AI in express mode with API key: {client._api_client.api_key[:5]}...{client._api_client.api_key[-5:]}"
    )

Using Vertex AI in express mode with API key: AIzaS...uWJxQ


In [7]:

MODEL_ID = "gemini-2.0-flash-001"

In [8]:
from arango import ArangoClient
from arango_datasets import Datasets


# Connect to database
db = ArangoClient(hosts="https://ca0d2b0035bc.arangodb.cloud:8529").db(username="root", password="Q77g2XUhBstTRLwlqWLK", verify=True)

# Connect to datasets
datasets = Datasets(db)

# List datasets
print(datasets.list_datasets())

# List more information about a particular dataset
print(datasets.dataset_info("SYNTHEA_P100"))

# Load a dataset
datasets.load("SYNTHEA_P100")
datasets = Datasets(db)

# List datasets
print(datasets.list_datasets())

# List more information about a particular dataset
print(datasets.dataset_info("SYNTHEA_P100"))

# Load a dataset
datasets.load("SYNTHEA_P100")

Output()

Output()

['IMDB_X', 'IMDB_PLATFORM', 'AMAZON_COMPUTER_PRODUCTS', 'AMAZON_PRODUCT_WITH_EMBEDDINGS', 'FAKE_HOMO_MULTICLASS', 'FAKE_HOMO_BINARY', 'FAKE_HETRO_LARGE_MULTICLASS', 'MAG', 'OPEN_INTELLIGENCE', 'OPEN_INTELLIGENCE_ANGOLA', 'FLIGHTS', 'CORA', 'DBLP', 'CVE', 'MTCARS', 'P2PSB', 'SCHAIN', 'ITSM_SAGE', 'ABIDE', 'CYBER', 'GAME_OF_THRONES', 'MARKET', 'SYNTHEA_P1000', 'SYNTHEA_P100']
{'files': 41, 'edges': [{'collection_name': 'patients_to_immunizations', 'files': ['https://arangodb-dataset-library-ml.s3.amazonaws.com/synthea_p100/edge_collection/patients_to_immunizations_3b4a9f4c6bbd45611715467144fd2b4d.data.json']}, {'collection_name': 'patients_to_encounters', 'files': ['https://arangodb-dataset-library-ml.s3.amazonaws.com/synthea_p100/edge_collection/patients_to_encounters_4d7fa7310d65555f8eb9525490d7be94.data.json']}, {'collection_name': 'encounters_to_observations', 'files': ['https://arangodb-dataset-library-ml.s3.amazonaws.com/synthea_p100/edge_collection/encounters_to_observations_ebfd1

Output()

Output()

...

Output()

Initializing collection 'patients_to_medications'


Output()

Output()

Output()

In [8]:
# 1. Load the NetworkX Graph into ArangoDB
# Reference: https://github.com/arangodb/nx-arangodb?tab=readme-ov-file#can-i-create-an-arangodb-graph-from-an-existing-networkx-graph
import networkx as nx
import nx_arangodb as nxadb

# Connect to the existing ArangoDB graph
G_adb = nxadb.Graph(name="SYNTHEA_P100", db=db)

print(G_adb)

[20:32:30 +0000] [INFO]: NetworkX-cuGraph is available.
INFO:nx_arangodb:NetworkX-cuGraph is available.


NameError: name 'db' is not defined

In [10]:
# 3. Print the degree of a Node

G_adb.degree(107)

2

In [11]:
# 1. Create the ArangoGraph LangChain wrapper
# Reference: https://api.python.langchain.com/en/latest/graphs/langchain_community.graphs.arangodb_graph.ArangoGraph.html
from langchain_community.graphs import ArangoGraph
arango_graph = ArangoGraph(db)
arango_graph

In [9]:
from IPython.display import Markdown, display
from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools import TavilySearchResults
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_vertexai import ChatVertexAI
from langchain_core.tools import tool

In [10]:
model = ChatGoogleGenerativeAI(model=MODEL_ID, temperature=0)
model.invoke("hello!")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-c8d4804b-b007-4575-b25e-ba1d87acb0c6-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [11]:
from langchain.chains import ArangoGraphQAChain


chain = ArangoGraphQAChain.from_llm(
    model, graph=arango_graph, verbose=True, allow_dangerous_requests=True
)


NameError: name 'arango_graph' is not defined

In [15]:
chain.run("most notable allergies?")

<ipython-input-15-cfdcf41f0939>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("most notable allergies?")




> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH allergies
FOR allergy IN allergies
SORT allergy.DESCRIPTION DESC
LIMIT 10
RETURN allergy

AQL Result:
[{'_key': '58', '_id': 'allergies/58', '_rev': '_jVTBIY---3', 'START': '1976-08-10', 'PATIENT': 'ab4c6a5c-2bfd-c83f-674d-723da21f2fae', 'ENCOUNTER': 'e768c4b2-81a5-80c2-8d25-cee821e18c45', 'CODE': 412071004, 'SYSTEM': 'SNOMED-CT', 'DESCRIPTION': 'Wheat (substance)', 'TYPE': 'allergy', 'CATEGORY': 'food'}, {'_key': '88', '_id': 'allergies/88', '_rev': '_jVTBIY--_V', 'START': '1996-07-20', 'PATIENT': '1172e1fa-e268-bf69-3243-b251c3b37e4c', 'ENCOUNTER': '2c624e0b-4773-d905-7b0c-ba9369e0c825', 'CODE': 412071004, 'SYSTEM': 'SNOMED-CT', 'DESCRIPTION': 'Wheat (substance)', 'TYPE': 'allergy', 'CATEGORY': 'food'}, {'_key': '127', '_id': 'allergies/127', '_rev': '_jVTBIYC--_', 'START': '1999-04-24', 'PATIENT': '61b35c4e-1267-f597-24f6-39b0defa71e0', 'ENCOUNTER': '2de9bcc4-f948-5955-b836-bc1c2cfa8992', 'CODE': 412071004, 'SYSTEM': '

'The most notable allergies include "Wheat (substance)" and "Tree pollen (substance)".'

In [12]:
from typing import Any, Dict
# 3. Define an example LangGraph Tool
# Note: This is not actually required for the Hackathon submission. This is just meant as a soft introduction to creating Tools in LangGraph

@tool
def favourite_fruit(query: str) -> str:
    """You are responsible for responding to being asked what your favourite fruit is.
    You must say Avocado!
    """
    return "Avocado!"

In [13]:
# 4. Define the Text to AQL Tool
# Reference: https://python.langchain.com/docs/integrations/graphs/arangodb/
# Reference: https://python.langchain.com/api_reference/community/chains/langchain_community.chains.graph_qa.arangodb.ArangoGraphQAChain.html


@tool
def text_to_aql_to_text(query: str) -> str:
    """
    This tool converts a natural language query into an AQL query,
    executes it against the ArangoDB graph, and translates the result
    back into natural language.
    """
    llm = model

    # Generate AQL code from the natural language query
    aql_code = llm.invoke(f"""
    I have an ArangoDB graph with the following schema: {arango_graph.schema}.
    I need to answer the following query: {query}.
    Generate the complete AQL code that will execute this query.
    Ensure that the result is stored in a variable called FINAL_RESULT.
    Only output AQL code (no additional commentary).
    Your AQL code:
    """).content.strip()

    print("Generated AQL code:")
    print(aql_code)
    print("-"*10)

    # Use ArangoGraphQAChain to execute the generated AQL query.
    chain = ArangoGraphQAChain.from_llm(
        llm=llm,
        graph=arango_graph,
        verbose=True,
        allow_dangerous_requests=True
    )
    result = chain.invoke(query)
    final_result = str(result["result"])

    # Formulate a natural language response based on FINAL_RESULT.
    aql_to_text = llm.invoke(f"""
        I have an ArangoDB graph with schema: {arango_graph.schema}.
        I executed the following AQL code to answer the query:
        ---
        {aql_code}
        ---
        The variable FINAL_RESULT is set to: {final_result}.
        Based on this information, generate a short and concise answer to the original query.
        Your response:
    """).content.strip()

    return aql_to_text

# Create the agent with both tools
def create_patient_analysis_agent():
    tools = [text_to_nx_algorithm_to_text, text_to_aql_to_text]
    agent = create_react_agent(model, tools)
    return agent

# Function to query the graph using the agent
def query_graph(query):
    agent = create_patient_analysis_agent()
    final_state = agent.invoke({"messages": [{"role": "user", "content": query}]})
    return final_state["messages"][-1].content

In [14]:
%env NX_CUGRAPH_AUTOCONFIG=True
import networkx

env: NX_CUGRAPH_AUTOCONFIG=True


In [15]:
import re
import networkx as nx
import os
import sys
from arango import ArangoClient
from arango_datasets import Datasets
import nx_arangodb as nxadb
from langchain_community.graphs import ArangoGraph
from IPython.display import Markdown, display
from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools import TavilySearchResults
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_vertexai import ChatVertexAI
from langchain_core.tools import tool
from langchain.chains import ArangoGraphQAChain
from langgraph.prebuilt import create_react_agent
import gradio as gr

# --- Colab Environment Setup ---
# Access API Keys from Colab Secrets
if "google.colab" in sys.modules:
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

    !pip install nx-cugraph-cu12  # Install cuGraph in Colab
    os.environ["NX_CUGRAPH_AUTOCONFIG"] = "True"


PROJECT_ID = "gem-creation"
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "true"

# Initialize LLM
MODEL_ID = "gemini-2.0-flash-001"  # Or another suitable model
# Fixed: Instantiated ChatGoogleGenerativeAI with google_api_key
model = ChatGoogleGenerativeAI(model=MODEL_ID, temperature=0.7, google_api_key = os.environ["GOOGLE_API_KEY"])

# --- ArangoDB Setup (Connect using Colab Secrets) ---

# Connect to database
db = ArangoClient(hosts="https://ca0d2b0035bc.arangodb.cloud:8529").db(username="root", password="Q77g2XUhBstTRLwlqWLK", verify=True)
datasets = Datasets(db)
datasets.load("SYNTHEA_P100")

# --- Graph Setup ---
# Connect to the existing ArangoDB graph
G_adb = nxadb.Graph(name="SYNTHEA_P100", db=db) #REAL
arango_graph = ArangoGraph(db=db)  # Create ArangoGraph wrapper #REAL

print(arango_graph.schema) # Print graph schema


# --- NetworkX Analysis Function (from your successful response) ---
def analyze_patient_network(db_connection, query, llm):
    """
    Performs a patient-focused analysis on an ArangoDB graph.

    Parameters:
    db_connection: The database connection
    query: A natural language query string
    llm: The language model to use for generating summaries

    Returns:
    A dictionary with analysis results
    """
    # Import nx at the beginning of the function to ensure it's available
    import networkx as nx
    # 1. Generate a multi-hop traversal
    def extract_condition_and_location(query_string):
        """Extracts condition and location from the query string."""
        # Look for condition and location pattern
        condition_location_match = re.search(r"with\s+(.+)\s+in\s+(.+)", query_string, re.IGNORECASE)

        # Alternative pattern - just look for location if no condition specified
        location_only_match = re.search(r"in\s+(.+)", query_string, re.IGNORECASE)

        if condition_location_match:
            condition = condition_location_match.group(1).strip()
            location = condition_location_match.group(2).strip()
            return condition, location
        elif location_only_match:
            # If only location is specified, return None for condition
            return None, location_only_match.group(1).strip()

        return None, None

    condition, location = extract_condition_and_location(query)
    # Check if we have at least a location
    if not location:
        return {
            "top_patients_with_context": "Query must specify at least a location (city/state).",
            "num_initial_patients": 0,
            "regional_patient_info": []
        }

    # 1a. Find initial patients using AQL
    # IMPORTANT: Access the correct database object
    # The error occurs because we're trying to use aql on a GraphDict object
    # Instead, we should use the underlying database object

    # First, get the total number of patients using the db object directly
    try:
        count_query = "RETURN LENGTH(FOR p IN patients RETURN p)"
        cursor = db_connection.aql.execute(count_query)  # Use db.aql instead of nx_graph.graph.aql
        total_number_patients = int(next(cursor))
    except Exception as e:
        print(f"Error counting patients: {e}")
        total_number_patients = 0  # Fallback value

    if not isinstance(total_number_patients, int):
        print(f"Error: total_number_patients must be an integer. Got {type(total_number_patients)}")
    print(f"Total number of patients: {total_number_patients}")

    # Modify the AQL query based on whether we have a condition or not
    if condition:
        aql_query = f"""
        LET target_condition = "{condition}"
        LET target_location = "{location}"

        FOR patient IN patients
            FILTER LOWER(patient.address) =~ LOWER(target_location)
            LET conditions = (
                FOR v, e IN 1..1 OUTBOUND patient patients_to_conditions
                    FILTER LOWER(v.condition) =~ LOWER(target_condition)
                    RETURN {{
                        condition: v.condition,
                    }}
            )
            FILTER LENGTH(conditions) > 0
            LET observations = (
                FOR v, e IN 1..3 OUTBOUND patient patients_to_observations
                    LET observation_time = DATE_TIMESTAMP(v.effectiveDateTime)
                    FILTER observation_time >= DATE_SUBTRACT(DATE_NOW(), 1, "year")
                    RETURN {{
                       observation: v.code,
                       value: v.valueQuantity.value
                    }}
            )
            RETURN {{
                patient_id: patient._key,
                conditions: conditions,
                observations: observations
            }}
        """
    else:
        # Query without condition filter
        aql_query = f"""
        LET target_location = "{location}"

        FOR patient IN patients
            FILTER LOWER(patient.address) =~ LOWER(target_location)
            LET conditions = (
                FOR v, e IN 1..1 OUTBOUND patient patients_to_conditions
                    RETURN {{
                        condition: v.condition,
                    }}
            )
            LET observations = (
                FOR v, e IN 1..3 OUTBOUND patient patients_to_observations
                    LET observation_time = DATE_TIMESTAMP(v.effectiveDateTime)
                    FILTER observation_time >= DATE_SUBTRACT(DATE_NOW(), 1, "year")
                    RETURN {{
                       observation: v.code,
                       value: v.valueQuantity.value
                    }}
            )

            RETURN {{
                patient_id: patient._key,
                conditions: conditions,
                observations: observations
            }}

        """

    cursor = db_connection.aql.execute(aql_query)
    regional_patient_info = [doc for doc in cursor]

    if not isinstance(regional_patient_info, list):
       print(f"Error: regional_patient_info must be a list. Got {type(regional_patient_info)}")

    sanitized_regional_patient_info = []
    for doc in regional_patient_info:
        sanitized_doc = {
            'patient_id': doc['patient_id'],
            'condition_count': len(doc['conditions']),
            'observation_count': len(doc['observations'])
        }
        sanitized_regional_patient_info.append(sanitized_doc)

    print(f"AQL Query Results (Sanitized): {sanitized_regional_patient_info}")

    # Check if we have any patients matching our criteria
    if not regional_patient_info:
        print("No patients found matching the criteria. Returning early.")
        return {
            "top_patients_with_context": f"No patients found with {condition} in {location}.",
            "num_initial_patients": total_number_patients,
            "regional_patient_info": []
        }

    # 2. Graph-based ranking
    # 2a. Create a Patient Multimodal Subgraph
    G = nx.Graph()
    for patient_data in regional_patient_info:
        patient_id = patient_data['patient_id']
        G.add_node(patient_id, type='patient')
        for condition_data in patient_data['conditions']:
            condition_name = condition_data['condition']
            G.add_node(condition_name, type='condition')
            G.add_edge(patient_id, condition_name, type='has_condition')
        for observation_data in patient_data['observations']:
            observation_name = observation_data['observation']
            observation_node_id = f"{observation_name}_{observation_data.get('value', 'N/A')}"
            G.add_node(observation_node_id, type='observation')
            G.add_edge(patient_id, observation_node_id, type='has_observation')

    # 2b. Calculate ranking (using NetworkX with cuGraph fallback)
    try:
        print("Calculating degree centrality with NetworkX...")
        degree_centrality = nx.degree_centrality(G)
        nx.set_node_attributes(G, degree_centrality, "degree_centrality")
        print("Successfully calculated degree centrality.")

    except Exception as e:
        print(f"Error calculating degree centrality: {e}. Trying alternative approach.")
        # Simple fallback if nx.degree_centrality fails
        degree_centrality = {}
        for node in G.nodes():
            degree_centrality[node] = len(list(G.neighbors(node))) / max(1, len(G) - 1)

    # Filter only patient nodes for the top ranking
    patient_centrality = {node: score for node, score in degree_centrality.items()
                         if G.nodes[node].get('type') == 'patient'}

    top_patients = sorted(patient_centrality.items(), key=lambda item: item[1], reverse=True)[:5]

    if not top_patients:
        print("No patients found in the graph. Check the graph construction.")
        return {
            "top_patients_with_context": f"Analysis completed but no patients with {condition} in {location} could be ranked.",
            "num_initial_patients": total_number_patients,
            "regional_patient_info": sanitized_regional_patient_info
        }

    # 3. LLM output generation
    prompt = f"""
    Based on patient data in {location}{' with ' + condition if condition else ''}:
    - Total patients in database: {total_number_patients}
    - Patients matching criteria: {len(regional_patient_info)}
    - Top 5 influential patients (by degree centrality in subgraph):
    {top_patients}

    Subgraph analysis includes patients, their conditions, and observations from the past year.
    Degree centrality measures how connected each patient is within this subgraph.

    Provide a human-readable summary, highlighting the top patients and why they rank highly in the network.
    """
    llm_result = llm.invoke(prompt)
    top_patients_with_context = llm_result.content

    FINAL_RESULT = {
        "top_patients_with_context": top_patients_with_context,
        "num_initial_patients": total_number_patients,
        "regional_patient_info": sanitized_regional_patient_info,
    }

    if not isinstance(FINAL_RESULT["top_patients_with_context"], str):
        print(f"Error: 'top_patients_with_context' must be a string. Got {type(FINAL_RESULT['top_patients_with_context'])}")
    if not isinstance(FINAL_RESULT["num_initial_patients"], int):
        print(f"Error: 'num_initial_patients' must be an integer. Got {type(FINAL_RESULT['num_initial_patients'])}")
    if not isinstance(FINAL_RESULT["regional_patient_info"], list):
        print(f"Error: 'regional_patient_info' must be a list. Got {type(FINAL_RESULT['regional_patient_info'])}")

    return FINAL_RESULT



# --- LLM Prompt for NetworkX Code Generation ---
def generate_nx_code(graph, query, llm):

  prompt = f"""
    I have a NetworkX Graph called `G_adb` derived from an ArangoDB database with the following schema: {arango_graph.schema}

    The graph contains many collections, including a "patients" collection, conditions and observations. I need to perform a patient‐focused analysis. Given this graph, combine graph analytics using networkx/cugraph together with LLM-generated text to perform retrieval augmented generation. The end results must be generated into human-readable text that an LLM should output! You are creating graphRAG functionality here.
    Given this graph, respond to the following natural language query, using NetworkX or CuGraph.
    Here is how you are to respond, create the code and store the data in a FINAL_RESULT dict.
    First - please run the bash %env NX_CUGRAPH_AUTOCONFIG=True in a codeblock.

    If the query asks for the "top 5 most influential patients with <condition> in <city/state>" or something similar, then, per graphRAG:

        1. Generate a multi-hop traversal: starting from patients in a region who have a specified condition perform a combined AQL with a limited traversals to gather information from conditions/observations/encounters.

           a. Find initial patients: First, use AQL to find all patients residing in the specified city/state AND who have the specified condition from the ‘conditions’ collection. BEFORE executing AQL query please capture how many patient nodes are included in the G_adb, call this variable total_number_patients and make sure it's the type int. The AQL result needs to include their 'condition', and all the observation values attached to them from the past year, limited to maximum of 3 hops using traversals. Capture this multi-hop AQL Query results in a python list format, call this query regional_patient_info (make sure to sanitize as to not output documents). Make sure all date and integer checks work by checking TYPE. Hint use patients_to_condition edges and patients_to_observations to limit dataset sizes! Remember to follow to generate and do the required actions: AQL TYPE is a STRING format, total num of node, number the results contain integer

           b. AQL visualization code here after result and total count

        2. Graph-based ranking - the generation happens next! You'll use that information from the multiple collections to perform node generation into an existing graph via adding new attributes and calculating a ranking/score of those nodes using cugraph, or else use NX, test and make function well to switch from NX->cugraph

           a. The multi-modal graph: From these regional_patient_info build a Patient Multimodal Subgraph, only add nodes and edges relevant and information derived from the data from regional_patient_info list for better computing power/node filtering

           b. Cugraph or NX compute: Use cugraph or NX (via automatic NX dispatch based on env variable) and calculate an approapriate algorithm scoring of those nodes for each patients! This relies in you calling NX function alone and trusting its ability for automatic allocation to NX cugraph with envar allocation (please check) then the other results are for the algorthrim to take effect

        3. LLM output generation to form text that would perform the human readble rag format and provide scores from scoring algorithms on those patients in human text

           a. Inject: The Prompt inject node rankings scores, results and total # of the result patient_info and other helpful data from prior steps, into a langchain google genai model LLM

           b. Return: Finally Generate the results from LLM with an inject so it returns a comprehensive reasonablble text
       Also keep in mind:
       Check whether graph conversion is working and test by including that function and include comments on if you pass the cugraph checking
       The total Number patients nodes BEFORE aql in total_number_patients is also checked!
       Sanitization on output data list
       Here is key: I also made sure now is the important part I'm enfored, then check it too, also make type = checks

    The final answer must be stored in the variable FINAL_RESULT as a dictionary with these keys (fill values appropriately, and assign keys and check the TYPES!!!) and is enforced:

       - "top_patients_with_context": (String) This variable name must contain the top patients found, together with some important characteristics that justify their influence - needs to be an LLM output
       - "num_initial_patients": (Integer) this must contain integer from of total num of inital patients counted before any of our aql - total_number_patients = #
       - "regional_patient_info": (List) Must contain LIST results from multi hop AQL - regional_patient_info - [] #LIST ENFORCED check TYPE!!!: LIST string for AQL value (hint TYPE check)!!! (data) . remember your code before when done right!!! So i enforce, that if does not make this, its type checking error code (since I cant see into execution error.) TYPE String for AQL! String remember from String!!! Sanitize the data in list
       - It is enforced all of that so please I remmebr you, as we have the ability now in final reuslt checking point before generation - to help our langchain - all key are checked to see its follow types too (code will now work well) so you remember for our next genaiton/model!

    Remember to make use to inject them all to get right response output so final injected promp works! Enforce TYPE also
    Sanitise the data using best coding with python!
    To clarify a good result = 1. Code follows step by step action - do not miss + can inject correct for my model with key words
                       2. Clean output and has sanity  and has inject-abilty for each variables into that LLM google langchin to build for output!
                           (so I get human result at the ned

    Remember make sanisze code, also add checking with your function, if NX -> cugraph passed as notes: 'Succesfull!' for check key step
    There are required part: all final injected are chekced by type  so must build to spec to use best! For test (a type checks, with best, to prevent)

    Make also ensure AQL works right/ well

    This will let  us follow up best in submission + test so I get great result
    Your code should follow these rules too or will be errored or break code too in the next stage: and get correct for result, key check points: TYPE MUST:
     Ensure key values like 'condition TYPE STRING ' for exmapel  and data follows. Also use other data when  approipaite not build new one so you reduce chances or eroro + key must follows - no miss that key
     Inject abililty, also follows as the prior code I followed. As you see. There must that abilties with the following

     Injected data with that key - since after this it'd injected direct and use must make use well (hence why my prior check does not miss in the next action to have in place if we folllows  all well

    To all steps!
     Also ensure follow + enabke (I test on Google COLB or will error since some codes do work) This is an require. Ensure all works as you will test - if fail test- this part will all wrong also follow this is cruaica

    In test,  we want to see output as follow (at key, the checks too - + show at comment the chekc part if key type pass after each code if so all works!) is for when to  I review to best give best follow at step!

    Helpful Resources you may be like:

          Make use again again to  Google search

        *  Check the prompt and result with google to follow our style

    This has 1-5 score 1 follows 5 breaks from instruc
    Again here what I wanted:: code folows well  with the TYPE I have to  and all + injected and then sanitize then work will (as example), make best for step for us!

     #DO AOT CREATE COMMETNS HERE THIS ARE ENFORCE CODE IS ENCORED (and no make sure in the right CODE spot place.) So it generates with my google LLM in injects right


        """

Output()

Initializing collection 'patients'


Output()

Output()

Initializing collection 'allergies'


...

Initializing collection 'encounters_to_imaging_studies'


Output()

Output()

[20:38:21 +0000] [INFO]: Graph 'SYNTHEA_P100' exists.
INFO:nx_arangodb:Graph 'SYNTHEA_P100' exists.
[20:38:22 +0000] [INFO]: Default node type set to 'allergies'
INFO:nx_arangodb:Default node type set to 'allergies'


{'Graph Schema': [{'graph_name': 'allergies', 'edge_definitions': [{'edge_collection': 'allergies_node_to_allergies_node', 'from_vertex_collections': ['allergies_node'], 'to_vertex_collections': ['allergies_node']}]}, {'graph_name': 'careplans', 'edge_definitions': [{'edge_collection': 'careplans_node_to_careplans_node', 'from_vertex_collections': ['careplans_node'], 'to_vertex_collections': ['careplans_node']}]}, {'graph_name': 'medications', 'edge_definitions': [{'edge_collection': 'medications_node_to_medications_node', 'from_vertex_collections': ['medications_node'], 'to_vertex_collections': ['medications_node']}]}, {'graph_name': 'procedures', 'edge_definitions': [{'edge_collection': 'procedures_node_to_procedures_node', 'from_vertex_collections': ['procedures_node'], 'to_vertex_collections': ['procedures_node']}]}, {'graph_name': 'supplies', 'edge_definitions': [{'edge_collection': 'supplies_node_to_supplies_node', 'from_vertex_collections': ['supplies_node'], 'to_vertex_collecti

In [16]:
@tool
def text_to_nx_algorithm_to_text(query: str) -> str:
    """This tool translates a natural language query into NetworkX code,
    executes it against the ArangoDB database, and returns a concise answer.
    It uses graph analysis techniques to identify influential patients in the database
    based on their connections to conditions and observations.
    """
    try:
        # Pass the db object to analyze_patient_network
        result = analyze_patient_network(db, query, model)
        # Check if result is a string (error message) or a dictionary
        if isinstance(result, str):
            return result
        elif isinstance(result, dict) and "top_patients_with_context" in result:
            return result["top_patients_with_context"]
        else:
            return f"Analysis completed but returned an unexpected format: {type(result)}"
    except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        error_message = f"Error in patient network analysis: {str(e)}\n{error_details}"
        print(error_message)
        return f"I encountered an error while analyzing the patient network: {str(e)}"

In [21]:
print(arango_graph.schema)

{'Graph Schema': [{'graph_name': 'allergies', 'edge_definitions': [{'edge_collection': 'allergies_node_to_allergies_node', 'from_vertex_collections': ['allergies_node'], 'to_vertex_collections': ['allergies_node']}]}, {'graph_name': 'careplans', 'edge_definitions': [{'edge_collection': 'careplans_node_to_careplans_node', 'from_vertex_collections': ['careplans_node'], 'to_vertex_collections': ['careplans_node']}]}, {'graph_name': 'medications', 'edge_definitions': [{'edge_collection': 'medications_node_to_medications_node', 'from_vertex_collections': ['medications_node'], 'to_vertex_collections': ['medications_node']}]}, {'graph_name': 'procedures', 'edge_definitions': [{'edge_collection': 'procedures_node_to_procedures_node', 'from_vertex_collections': ['procedures_node'], 'to_vertex_collections': ['procedures_node']}]}, {'graph_name': 'supplies', 'edge_definitions': [{'edge_collection': 'supplies_node_to_supplies_node', 'from_vertex_collections': ['supplies_node'], 'to_vertex_collecti

In [17]:
# 6. Create the Agentic Application

# Note: Consider implementing a hybrid tool that combines both AQL & NetworkX Algorithms!
# Import create_react_agent
from langgraph.prebuilt import create_react_agent

tools = [text_to_aql_to_text, text_to_nx_algorithm_to_text]

def query_graph(query):
    llm = model
    app = create_react_agent(llm, tools)
    final_state = app.invoke({"messages": [{"role": "user", "content": query}]})
    return final_state["messages"][-1].content

In [25]:
query_graph("The locations of patients with Diabetes?")

'I cannot provide you with the locations of patients with diabetes. I do not have the ability to access specific patient locations or other protected health information.'

In [18]:
def query_graph_with_router(query):
    """
    Routes queries to the appropriate tool based on pattern matching
    rather than relying on the agent's tool selection.
    """
    # Check if the query is looking for influential patients with a condition in a location
    if any(keyword in query.lower() for keyword in ["influential", "important", "top"]) and \
       (("with" in query.lower() and "in" in query.lower()) or
        ("patients" in query.lower() and "in" in query.lower())):
        # This is a network analysis query, use the NetworkX tool directly
        print("Routing to NetworkX analysis tool...")
        return text_to_nx_algorithm_to_text.invoke(query)  # Use invoke() instead of __call__
    else:
        # For other types of queries, use the agent to decide
        print("Using agent for tool selection...")
        agent = create_patient_analysis_agent()
        final_state = agent.invoke({"messages": [{"role": "user", "content": query}]})
        return final_state["messages"][-1].content

In [22]:
query_graph_with_router("Find the top 5 most influential patients in California?")

Routing to NetworkX analysis tool...
Total number of patients: 124
AQL Query Results (Sanitized): []
No patients found matching the criteria. Returning early.


'No patients found with None in California?.'

In [ ]:
import gradio as gr

gr.Interface(fn=query_graph, inputs="text", outputs="text").launch(share=True)